In [1]:
# lstm_lstm: emotion recognition from speech= lstm, text=lstm
# created for ATSIT paper 2020
# coded by Bagus Tris Atmaja (bagus@ep.its.ac.id)

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from gensim.models.keyedvectors import KeyedVectors

Using TensorFlow backend.
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/envs/ser36/lib/python3.6/sit

In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:

# text feature and labels
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
# x_train_text = np.load(path+'x_train_text.npy')
# vad = np.load(path+'y_egemaps.npy')
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')

data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)
# standardization
scaled_feature = False

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)
EMBEDDING_DIM = 300
word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

file_loc = '../../dimensionalSM/data/GoogleNews-vectors-negative300.bin'
print (file_loc)

word_vectors = KeyedVectors.load_word2vec_format(file_loc, binary=True)

g_word_embedding_matrix = {}
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=nb_words:
        continue
    try:
        gembedding_vector = word_vectors[word]
        g_word_embedding_matrix[i] = gembedding_vector
    except KeyError:
        g_word_embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), EMBEDDING_DIM)
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))
del(word_vectors)



scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# split train/test

earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)



Found 3437 unique tokens
../../dimensionalSM/data/GoogleNews-vectors-negative300.bin
G Null word embeddings: 1


In [4]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [5]:
split = 7869

In [6]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)
    

    # combined model
    model_combined = concatenate([model_speech, model_text])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


#model_1.summary()

In [7]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_train_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_train_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(30):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('speech_textW2V_multi7869.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 50s 8ms/step - loss: 1.5608 - v_loss: 0.6368 - a_loss: 0.4107 - d_loss: 0.5129 - v_ccc: 0.3628 - a_ccc: 0.5893 - d_ccc: 0.4871 - val_loss: 2.2989 - val_v_loss: 0.7562 - val_a_loss: 0.7900 - val_d_loss: 0.7556 - val_v_ccc: 0.2502 - val_a_ccc: 0.2093 - val_d_ccc: 0.2416
Epoch 2/50
6295/6295 [==============================] - 47s 8ms/step - loss: 1.1176 - v_loss: 0.3778 - a_loss: 0.3197 - d_loss: 0.4198 - v_ccc: 0.6221 - a_ccc: 0.6803 - d_ccc: 0.5800 - val_loss: 1.7577 - val_v_loss: 0.6217 - val_a_loss: 0.5661 - val_d_loss: 0.5759 - val_v_ccc: 0.3812 - val_a_ccc: 0.4365 - val_d_ccc: 0.4246
Epoch 3/50
6295/6295 [==============================] - 48s 8ms/step - loss: 0.9838 - v_loss: 0.3085 - a_loss: 0.3004 - d_loss: 0.3747 - v_ccc: 0.6916 - a_ccc: 0.6995 - d_ccc: 0.6251

Epoch 8/50
6295/6295 [==============================] - 47s 7ms/step - loss: 0.6479 - v_loss: 0.1767 - a_loss: 0.2160 - d_loss: 0.2552 - v_ccc: 0.8232 - a_ccc: 0.7840 - d_ccc: 0.7449 - val_loss: 1.5741 - val_v_loss: 0.5491 - val_a_loss: 0.4611 - val_d_loss: 0.5857 - val_v_ccc: 0.4615 - val_a_ccc: 0.5442 - val_d_ccc: 0.4202
Epoch 9/50
6295/6295 [==============================] - 46s 7ms/step - loss: 0.5982 - v_loss: 0.1628 - a_loss: 0.1972 - d_loss: 0.2380 - v_ccc: 0.8372 - a_ccc: 0.8026 - d_ccc: 0.7620 - val_loss: 1.5386 - val_v_loss: 0.5420 - val_a_loss: 0.4553 - val_d_loss: 0.5677 - val_v_ccc: 0.4688 - val_a_ccc: 0.5506 - val_d_ccc: 0.4420
Epoch 10/50
6295/6295 [==============================] - 46s 7ms/step - loss: 0.5474 - v_loss: 0.1512 - a_loss: 0.1828 - d_loss: 0.2137 - v_ccc: 0.8490 - a_ccc: 0.8172 - d_ccc: 0.7864 - val_loss: 1.6018 - val_v_loss: 0.5691 - val_a_loss: 0.4521 - val_d_loss: 0.6122 - val_v_ccc: 0.4412 - val_a_ccc: 0.5551 - val_d_ccc: 0.4019
Epoch 11/50
6295/6295 [=

2170/2170 [==============================] - 4s 2ms/step
[0.41981977224349976, 0.5557342767715454, 0.4631769359111786]
2
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 51s 8ms/step - loss: 1.5354 - v_loss: 0.6062 - a_loss: 0.4156 - d_loss: 0.5133 - v_ccc: 0.3936 - a_ccc: 0.5844 - d_ccc: 0.4865 - val_loss: 2.0126 - val_v_loss: 0.6741 - val_a_loss: 0.7002 - val_d_loss: 0.6430 - val_v_ccc: 0.3325 - val_a_ccc: 0.2999 - val_d_ccc: 0.3550
Epoch 2/50
6295/6295 [==============================] - 47s 7ms/step - loss: 1.1161 - v_loss: 0.3752 - a_loss: 0.3260 - d_loss: 0.4153 - v_ccc: 0.6249 - a_ccc: 0.6744 - d_ccc: 0.5846 - val_loss: 1.6131 - val_v_loss: 0.6057 - val_a_loss: 0.4974 - val_d_loss: 0.5215 - val_v_ccc: 0.3985 - val_a_ccc: 0.5077 - val_d_ccc: 0.4808
Epoch 3/50
6295/6295 [==============================] - 48s 8ms/step - loss: 0.9925 - v_loss: 0.3088 - a_loss: 0.3042 - d_loss: 0.3794 - v_ccc: 0.6910 - a_ccc: 0.6958 - d_ccc: 0.620

6295/6295 [==============================] - 47s 7ms/step - loss: 0.6537 - v_loss: 0.1765 - a_loss: 0.2202 - d_loss: 0.2565 - v_ccc: 0.8234 - a_ccc: 0.7796 - d_ccc: 0.7433 - val_loss: 1.4941 - val_v_loss: 0.5513 - val_a_loss: 0.4322 - val_d_loss: 0.5279 - val_v_ccc: 0.4585 - val_a_ccc: 0.5702 - val_d_ccc: 0.4772
Epoch 9/50
6295/6295 [==============================] - 47s 7ms/step - loss: 0.6117 - v_loss: 0.1624 - a_loss: 0.2055 - d_loss: 0.2442 - v_ccc: 0.8375 - a_ccc: 0.7950 - d_ccc: 0.7558 - val_loss: 1.6287 - val_v_loss: 0.5647 - val_a_loss: 0.4950 - val_d_loss: 0.5875 - val_v_ccc: 0.4439 - val_a_ccc: 0.5061 - val_d_ccc: 0.4213
Epoch 10/50
6295/6295 [==============================] - 47s 7ms/step - loss: 0.5582 - v_loss: 0.1477 - a_loss: 0.1945 - d_loss: 0.2160 - v_ccc: 0.8523 - a_ccc: 0.8055 - d_ccc: 0.7840 - val_loss: 1.5907 - val_v_loss: 0.5595 - val_a_loss: 0.4673 - val_d_loss: 0.5870 - val_v_ccc: 0.4498 - val_a_ccc: 0.5385 - val_d_ccc: 0.4210
Epoch 11/50
6295/6295 [============

6295/6295 [==============================] - 47s 7ms/step - loss: 0.8950 - v_loss: 0.2635 - a_loss: 0.2820 - d_loss: 0.3494 - v_ccc: 0.7367 - a_ccc: 0.7180 - d_ccc: 0.6503 - val_loss: 1.5773 - val_v_loss: 0.6163 - val_a_loss: 0.4287 - val_d_loss: 0.5494 - val_v_ccc: 0.3908 - val_a_ccc: 0.5779 - val_d_ccc: 0.4540
Epoch 5/50
6295/6295 [==============================] - 47s 7ms/step - loss: 0.8159 - v_loss: 0.2282 - a_loss: 0.2667 - d_loss: 0.3211 - v_ccc: 0.7717 - a_ccc: 0.7335 - d_ccc: 0.6788 - val_loss: 1.5531 - val_v_loss: 0.5683 - val_a_loss: 0.4376 - val_d_loss: 0.5639 - val_v_ccc: 0.4380 - val_a_ccc: 0.5700 - val_d_ccc: 0.4390
Epoch 6/50
6295/6295 [==============================] - 47s 8ms/step - loss: 0.7607 - v_loss: 0.2104 - a_loss: 0.2482 - d_loss: 0.3020 - v_ccc: 0.7897 - a_ccc: 0.7519 - d_ccc: 0.6978 - val_loss: 1.5668 - val_v_loss: 0.5593 - val_a_loss: 0.4518 - val_d_loss: 0.5808 - val_v_ccc: 0.4495 - val_a_ccc: 0.5603 - val_d_ccc: 0.4234
Epoch 7/50
6295/6295 [==============

...

Epoch 13/50
6295/6295 [==============================] - 47s 7ms/step - loss: 0.4515 - v_loss: 0.1224 - a_loss: 0.1559 - d_loss: 0.1738 - v_ccc: 0.8777 - a_ccc: 0.8442 - d_ccc: 0.8266 - val_loss: 1.5833 - val_v_loss: 0.5422 - val_a_loss: 0.4898 - val_d_loss: 0.5759 - val_v_ccc: 0.4707 - val_a_ccc: 0.5126 - val_d_ccc: 0.4334
Epoch 14/50
2170/2170 [==============================] - 5s 2ms/step
[0.41985219717025757, 0.5633007884025574, 0.4896725118160248]
21
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 59s 9ms/step - loss: 1.5440 - v_loss: 0.6009 - a_loss: 0.4190 - d_loss: 0.5241 - v_ccc: 0.3990 - a_ccc: 0.5812 - d_ccc: 0.4757 - val_loss: 2.2411 - val_v_loss: 0.6994 - val_a_loss: 0.8265 - val_d_loss: 0.7255 - val_v_ccc: 0.3084 - val_a_ccc: 0.1765 - val_d_ccc: 0.2740
Epoch 2/50
6295/6295 [==============================] - 47s 7ms/step - loss: 1.0940 - v_loss: 0.3706 - a_loss: 0.3163 - d_loss: 0.4072 - v_ccc: 0.6295 - a_ccc: 0.6837 

6295/6295 [==============================] - 47s 7ms/step - loss: 0.6138 - v_loss: 0.1692 - a_loss: 0.2013 - d_loss: 0.2432 - v_ccc: 0.8307 - a_ccc: 0.7986 - d_ccc: 0.7569 - val_loss: 1.5738 - val_v_loss: 0.5427 - val_a_loss: 0.4727 - val_d_loss: 0.5809 - val_v_ccc: 0.4659 - val_a_ccc: 0.5365 - val_d_ccc: 0.4237
Epoch 10/50
6295/6295 [==============================] - 47s 8ms/step - loss: 0.5804 - v_loss: 0.1581 - a_loss: 0.1947 - d_loss: 0.2277 - v_ccc: 0.8422 - a_ccc: 0.8052 - d_ccc: 0.7722 - val_loss: 1.5618 - val_v_loss: 0.5553 - val_a_loss: 0.4624 - val_d_loss: 0.5687 - val_v_ccc: 0.4528 - val_a_ccc: 0.5445 - val_d_ccc: 0.4409
Epoch 11/50
6295/6295 [==============================] - 46s 7ms/step - loss: 0.5327 - v_loss: 0.1460 - a_loss: 0.1804 - d_loss: 0.2061 - v_ccc: 0.8541 - a_ccc: 0.8194 - d_ccc: 0.7938 - val_loss: 1.6077 - val_v_loss: 0.5542 - val_a_loss: 0.4962 - val_d_loss: 0.5894 - val_v_ccc: 0.4566 - val_a_ccc: 0.5108 - val_d_ccc: 0.4249
Epoch 12/50
6295/6295 [===========

Epoch 19/50
2170/2170 [==============================] - 5s 2ms/step
[0.4211485683917999, 0.5461525321006775, 0.47786933183670044]
24
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 60s 10ms/step - loss: 1.5236 - v_loss: 0.5890 - a_loss: 0.4082 - d_loss: 0.5260 - v_ccc: 0.4108 - a_ccc: 0.5917 - d_ccc: 0.4739 - val_loss: 1.9026 - val_v_loss: 0.6794 - val_a_loss: 0.6554 - val_d_loss: 0.5784 - val_v_ccc: 0.3270 - val_a_ccc: 0.3474 - val_d_ccc: 0.4230
Epoch 2/50
6295/6295 [==============================] - 47s 7ms/step - loss: 1.0999 - v_loss: 0.3693 - a_loss: 0.3179 - d_loss: 0.4121 - v_ccc: 0.6305 - a_ccc: 0.6819 - d_ccc: 0.5877 - val_loss: 1.6000 - val_v_loss: 0.6045 - val_a_loss: 0.4813 - val_d_loss: 0.5295 - val_v_ccc: 0.4020 - val_a_ccc: 0.5251 - val_d_ccc: 0.4728
Epoch 3/50
6295/6295 [==============================] - 47s 7ms/step - loss: 0.9739 - v_loss: 0.3007 - a_loss: 0.2985 - d_loss: 0.3750 - v_ccc: 0.6996 - a_ccc: 0.7015 

6295/6295 [==============================] - 46s 7ms/step - loss: 0.5327 - v_loss: 0.1446 - a_loss: 0.1783 - d_loss: 0.2096 - v_ccc: 0.8554 - a_ccc: 0.8217 - d_ccc: 0.7903 - val_loss: 1.5769 - val_v_loss: 0.5649 - val_a_loss: 0.4523 - val_d_loss: 0.5777 - val_v_ccc: 0.4449 - val_a_ccc: 0.5471 - val_d_ccc: 0.4310
Epoch 12/50
6295/6295 [==============================] - 47s 7ms/step - loss: 0.4957 - v_loss: 0.1357 - a_loss: 0.1686 - d_loss: 0.1914 - v_ccc: 0.8643 - a_ccc: 0.8315 - d_ccc: 0.8085 - val_loss: 1.6162 - val_v_loss: 0.5499 - val_a_loss: 0.4886 - val_d_loss: 0.5991 - val_v_ccc: 0.4605 - val_a_ccc: 0.5157 - val_d_ccc: 0.4075
Epoch 13/50
6295/6295 [==============================] - 47s 7ms/step - loss: 0.4665 - v_loss: 0.1284 - a_loss: 0.1602 - d_loss: 0.1778 - v_ccc: 0.8715 - a_ccc: 0.8398 - d_ccc: 0.8222 - val_loss: 1.5641 - val_v_loss: 0.5443 - val_a_loss: 0.4689 - val_d_loss: 0.5668 - val_v_ccc: 0.4662 - val_a_ccc: 0.5322 - val_d_ccc: 0.4376
Epoch 14/50
6295/6295 [===========

6295/6295 [==============================] - 47s 8ms/step - loss: 0.7624 - v_loss: 0.2119 - a_loss: 0.2469 - d_loss: 0.3037 - v_ccc: 0.7881 - a_ccc: 0.7530 - d_ccc: 0.6964 - val_loss: 1.5255 - val_v_loss: 0.5512 - val_a_loss: 0.4376 - val_d_loss: 0.5582 - val_v_ccc: 0.4576 - val_a_ccc: 0.5696 - val_d_ccc: 0.4473
Epoch 7/50
6295/6295 [==============================] - 47s 7ms/step - loss: 0.6963 - v_loss: 0.1895 - a_loss: 0.2316 - d_loss: 0.2749 - v_ccc: 0.8104 - a_ccc: 0.7683 - d_ccc: 0.7250 - val_loss: 1.5405 - val_v_loss: 0.5487 - val_a_loss: 0.4492 - val_d_loss: 0.5618 - val_v_ccc: 0.4609 - val_a_ccc: 0.5544 - val_d_ccc: 0.4443
Epoch 8/50
6295/6295 [==============================] - 47s 8ms/step - loss: 0.6508 - v_loss: 0.1769 - a_loss: 0.2170 - d_loss: 0.2572 - v_ccc: 0.8230 - a_ccc: 0.7833 - d_ccc: 0.7429 - val_loss: 1.5230 - val_v_loss: 0.5404 - val_a_loss: 0.4513 - val_d_loss: 0.5564 - val_v_ccc: 0.4706 - val_a_ccc: 0.5546 - val_d_ccc: 0.4518
Epoch 9/50
6295/6295 [==============